## Transfer Learning: Using Keras API: Fine-Tuning Pretrained Deep Learning Networks with Data Augmentation 

This notebook contains Transfer learning code for extracting features using the fine-tuning method of pre-trained  networks. In fine-tuning a pre-trained network, we assume that our custom dataset is small (less than 1000 images per class) and not similar to the dataset that the pre-trained netwoks were trained on (ImageNet dataset in this case). Fine-tuning is complimentary to using a pre-trained model/network for feature extraction. Fine-tuning makes slight adjustments to the abstract representations learned(in the deeper layers) by the pre-trained model to make them more relevant to the custom problem being addressed. In this fine tuning method, we will also use Keras data augmentation methods to randomly transform the original dataset on the fly during model training (which is analogous to generating additional training data). It is advisable to run this code ONLY on a GPU as it is too expensive to run on a CPU.

## Fine Tuning Procedure

This procedure is only for the fine tuning process and does not include data pre-processin tasks:

1) Load the convolutional base of the pre-trained network/model <br><br>
2) Flatten the convolutional base outputs (before feeding them to the densely   
  connected classifier <br><br>
3) Add a custom densely connected classifier on top of the flattened convolutional base  
   model <br><br>
4) Freeze the convolutional base model <br><br>
5) Compile the model <br><br>
6) Train the model (end to end training with data augmentation)--<br>- i.e. train the added part--<br> [Same as feature extraction with data augmentation up to this step] <br> This step trains  the classifier that was added in step 3. You can also save the model after this step to act as the feature extraction part of the model <br><br>
7) Unfreeze some layers in the convolutional base of the network (This step is achieved by unfreezing the whole convolutional base followed by the freezing of some individual layers inside the convolutional base) 
 <br><br>
  
8) Re-compile the model <br><br> It is very important to recompile the model after changing the weight trainability of the layers (unfreezing), otherwise the changes made in step 7 will be ignored
  
9) Jointly train these (unfrozen) layers with the part added in step 3 <br>

Had, at this point, the added custom classifier not been trained in step 6, the error signal propagated through the network during training in this step would be too large, leading to the destruction of the feature representations that were previously learned by the layers being fine-tuned.


10) Save the model

## Import the Necessary Libraries/Packages

In [ ]:
# Import the necessary libraries

from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import ResNet50
from keras.applications import InceptionV3
from keras.applications import Xception
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions 
from keras.applications.inception_v3 import preprocess_input
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.layers import Input, Flatten, Dense, Dropout
from keras import optimizers
from keras.optimizers import Adam
from keras.layers import merge, Input
from keras import models
from keras import layers
from keras.models import Model
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator    # input preprocessing done in the ImageDataGenerator
import numpy as np
from sklearn.utils import shuffle 
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import itertools
from imutils import paths
import random
import os
import shutil
from pathlib import Path
import time

## Data Preprocessing


See data processing code in a separate Jupyter Notebook


## Constant Variables (Configuration Cell)

In [ ]:
# initialize the path to the main input directory of images
MAIN_INPUT_DATASET = "C:\\Path\\to\\Main\\Input\\Directory"

# initialize the base path to the directory that will contain
# the images after computing the training, validation, and testing split
BASE_PATH = "C:\\Path\\to\\Base\\Path"

# derive the the paths to the training, validation, and testing directories
TRAIN_PATH = os.path.sep.join([BASE_PATH, "training"])
VAL_PATH = os.path.sep.join([BASE_PATH, "validation"])
TEST_PATH = os.path.sep.join([BASE_PATH, "testing"])

# examine whether the derived paths are correct
print(TRAIN_PATH)
print(VAL_PATH)
print(TEST_PATH)

# set the percentage of data that will be used for training (training split)
TRAIN_SPLIT = 0.8
 
# set the percentage of validation data that will used for validation during training
# the validation data set will be a percentage of the training data/i.e it will be split from the training data
# here val is set at 25% split from the 80% of training data, i.e. 20% of the total dataset
# we end up with train:test:validation splits ratios equal to= 60:20:20
VAL_SPLIT = 0.25


## Project Structure (Training, Validation, and Testing Directory Structure)

In [ ]:
image = mpimg.imread("C:\\Path\\to\\Directory\\Structure\\Image\\directory_tree.png")
plt.imshow(image)
plt.show()

## Split/Partition the Data into Training, Validation, and Testing Sets

See separate code for partitioning data

## Deep Learning Model Training for Crop Diseases Classification

The model training code starts from here.

In [ ]:
# Print out the total number of images in each of the training, validation, 
# and test splits/directories

ntotalTrain = len(list(paths.list_images(TRAIN_PATH)))
ntotalVal = len(list(paths.list_images(VAL_PATH)))
ntotalTest = len(list(paths.list_images(TEST_PATH)))

print('The total number of training images =', ntotalTrain)
print('The total number of validation images =', ntotalVal)
print('The total number of testing images =', ntotalTest)

## Define the Names of the Pre-Trained Networks 

In [ ]:
# define a list of Keras pretrained model names 
# the model names to use in this code should be chosen from the following list
MODELS = ["vgg16", "vgg19", "resnet50", "inception", "xception"] 

## Define the Keras Pre-Trained Model Names

In [ ]:
while True:
    Name = str(input('Enter the Model you want to run today:...:'))
    if Name not in MODELS:
        print("The model name you entered is not in one of the names in the Models list. Please select one from the list")
        print("Please enter one of the following model names: vgg16, vgg19, resnet50, inception, xception")
        # return to the start of the loop
        continue 
    else:
        # exit the loop
        print("You have entered a valid model name")
        break
        
if Name in ("vgg16", "vgg19", "resnet50"):
    input_shape, target_size = (256, 256, 3), (256, 256)
      
else:
    input_shape, target_size = (256, 256, 3), (256, 256)
    
Network_name = Name                            
base = Network_name + "_conv_base"

# check to see whether the inputs are correct
print("The name of the chosen Network is...:", Network_name)
print("The input shape of the chosen Network is...:", input_shape)
print("The target_size of the chosen Network is...:", target_size)
print("The convolutional base of the chosen Network is defined as...:", base)

print('{}{}{}{}'.format("The network chosen is...: ", Network_name, ", and the conv. base of the Network is...:", base))    
print('{}{}{}{}'.format("The target_size for...: ", Network_name, ", is...:", target_size)) 
        

## Set Training Parameters

In [ ]:
# set the batch size
batch_size = 30

# set the number of epochs
epochs = 5
   


## Apply Data Augmentation 
<br> The Keras ImageDataGenerator yields batches of images from disk which eliminates <br>
the need for holding the entire dataset in memory (good, especially if we are limited in memory)

## Initialize the Data Generators 

In [ ]:
# initialize the training data augmentation object
train_datagen = ImageDataGenerator(rescale=1./255,                                 
                            rotation_range=40,                               
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

# initialize the validation (also used for testing) data augmentation object
# Validation and test datasets are not augmented

val_datagen = ImageDataGenerator(rescale=1./255,                                  
                                )                                             


## Initialize the Data Generators (Using .flow_from_directory Method)

In [ ]:
# initialize the training generator
train_generator = train_datagen.flow_from_directory(TRAIN_PATH,
                                                    class_mode="categorical",
                                                    target_size=target_size,
                                                    color_mode="rgb", 
                                                    batch_size=batch_size,
                                                    shuffle=shuffle)

# initialize the validation generator
val_generator = val_datagen.flow_from_directory(VAL_PATH,
                                                class_mode="categorical",
                                                target_size=target_size,
                                                color_mode="rgb",   
                                                batch_size=batch_size,
                                                shuffle=False)  # validation dataset should not be shuffled

# initialize the testing generator
test_generator = val_datagen.flow_from_directory(TEST_PATH,
                                                 class_mode="categorical",  
                                                 target_size=target_size,                          
                                                 color_mode="rgb",   
                                                 batch_size=batch_size,
                                                 shuffle=False)      # testing dataset should not be shuffled 
                                                                     # so as to keep data in same order as the labels
                                                                 

## Handling Imbalanced Datasets

In [ ]:
# compute the class weights if the custom dataset is imbalanced with some 
# classes having more data than others
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)

## Load the Convolutional Base (of the Pre-trained Model/Network)

In [ ]:
#Get the feature extraction part of the network pre-trained on ImageNet (convolutional base)

# load the selected model/Network convolutional base weights

input_shape = input_shape

if Network_name == "vgg16":
    base = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

elif Network_name == "vgg19":
    base = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

elif Network_name == "resnet50":
    base = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

elif Network_name == "inception":
    base = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

elif Network_name == "xception":
    base = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

else:
    babe = None
    
print ("[INFO] Successfully loaded the convolutional base of the", Network_name, "Pretrained Network")
print('{} {}'.format("Shown below is the convolutional base summary for: ", Network_name))

print ("[INFO] Below is the summary of the convolutional base of: ", Network_name)
print(base.summary())

## Add a Custom Densely-Connected Classifier on Top of the Convolutional Base

In [ ]:
# Create our custom classification layer
num_classes = 4    # change this number depending on the number of training classes

# Create a custom Classifier Model 
x = base.output
x = Flatten()(x)                       # Flatten convolutional base 
x = Dense(256, activation='relu')(x)   # add a dense layer with ReLu activation
x = Dropout(0.5)(x)                    # add dropout
predictions = Dense(num_classes, activation='softmax')(x) # add a classification layer

# the following is the model that will be trained
model = Model(inputs=base.input, outputs=predictions)

print(model.summary())                # print the new whole model summary

## Freeze the Convolutional Base

Freeze the convolutional base before compiling and training the model. This prevents the weights of the convolutional base from being updated during training, otherwise the representations previously learned by the convolutional base will be modified during training, i.e, freezing prevents the representations previously learned by the convolutional base from being modified during training. Since the dense layers added on top are randomly initialized, freezing prevents very large weight updates from being propagated through the network as this would destroy the previously learned representations. 

In [ ]:
# check the effect of freezing the convolutional base on the number of trainable weights
print('This is the number of trainable weights before freezing the convolutional base:', len(model.trainable_weights))

# freeze all the layers except the dense (fully connected) layers
for layer in base.layers:
    layer.trainable = False
    
print('This is the number of trainable weights after freezing the convolutional base:', len(model.trainable_weights))

In [ ]:
# We can now check the trainable status of the individual layers
# in the whole model
for i, layer in enumerate(model.layers):
   print(i, layer.trainable)

## Compile the Model

In [ ]:
# compile the model to ensure the 'trainable' changes have taken effect
model.compile(optimizer=optimizers.Adam(lr=2e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

## Model Summary

In [ ]:
# Check the number of trainable parameters 
# this should show in the summary after compiling the model
model.summary()

## Train the Model
It is essential to train the model at this stage (before fine-tuning some layers in the base) because the top layers of the convolutional base can only be fine-tuned when the classifier on top has been trained, otherwise the representations previously learned by the layers being fine-tuned will be destroyed by the very large error signal that will be propagated (by the untrained classifier) through the network during training.

In [ ]:
# time the training process
%%time                                            
history = model.fit_generator(train_generator,
                    steps_per_epoch=ntotalTrain // batch_size,
                    epochs=epochs,
                    validation_data= val_generator,
                    validation_steps=ntotalVal // batch_size,
                    class_weight=class_weights)

## Save the Model

In [ ]:

model.save("C:\\Path\\to\\TrainedModel.h5")

# You can also save model without the optimizer to avoid the optimizer error shown when loading the trained model later or
# in a different environment

model.save("C:\\Path\\to\\TrainedModel_1.h5, include_optimizer= False")


All the steps above are used for feature extraction with data augmentation type of transfer learning. The following steps can be added to turn this feature extraction transfer learning into a fine-tuning type of transfer learning. BUT we need to move the steps for saving the model and ploting the results to the end of the process. We do not need to save or plot the intermedairy resulsts. WE CAN also reduce the number of EPOCHS that we used in the feature extraction steps (just run for enough epochs to ensure that the top/classification layers are well trained)

## Evaluate the Model Performance/ Plot the results
## IMPORTANT

We can also evaluate the model performance tt this point if we want. This will essentially turn the code in this Jupyter Notebook into both a feature extractor (with data augmentation) and a fine-tuning model. This is because at this point, all the above steps are for feature extraction transfer learning. Combining the above steps with the steps that follow below wil give a final performance for fine-tuning the model. This kills two birds with one stone.  

## Plot the Training Results¶

In [ ]:
# This code will plot the loss and accuracy values during model training

# Get values that were specified during model compilation which are saved in the 
# history object
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get the number of epochs from the values in the 'acc' list
epochs = range(1, len(acc) + 1)

# Training and validation accuracy plot [Accuracy at each epoch]
plt.plot(epochs, acc, 'b', label='Training acc')       
plt.plot(epochs, val_acc, 'r', label='Validation acc')  
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='center right')
plt.figure()

# Training and validation loss plot [Loss at each epoch]
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='center right')
plt.show()


## Process Check

In [ ]:
# to determine the crop diseases were are training
# split the BASE PATH into it's parts
BASE_PATH_LIST = BASE_PATH.split(os.sep)
print(BASE_PATH_LIST)
CropName = BASE_PATH_LIST[-1] 

# extract the crop of interest from the path and print its name
print('{}{}'.format("This network is learning to classify the diseases of the crop known as:..", CropName)

# list the target names (diseases being classified) for each crop in alphaneumeric order 
      
if BASE_PATH_LIST[-1] == "Apples":
      target_names = ['AppleScab', 'BlackRot', 'CedarAppleRust', 'Healthy']
elif BASE_PATH_LIST[-1] == "Grapes":
       target_names = ['Healthy', 'GrapeBlackMeasles', 'GrapeBlackRot', 'GrapeLeafBlight']      
elif BASE_PATH_LIST[-1] == "Tomatoes": 
      target_names = ['Healthy', 'TomatoBacterialSpot', 'TomatoEarlyBlight', 'TomatoLateBlight']
elif BASE_PATH_LIST[-1] == "Corn":
      ['CornCommonRust', 'CornGrayLeafSpot', 'CornNorthernLeafBlight', 'Healthy'] 
else:
      target_names = None
      
print('{}{}'.format("The current target names of the diseases are:..", target_names)

## Evaluate Model on the Testing Dataset/ Make Inferences from Unseen Data

In [ ]:
# First, reset the testing generator before using the trained model to make predictions
# on the test data
print("[INFO] Now evaluating the trained network...")
test_generator.reset()
predictions = model.predict_generator(test_generator,
                                   steps=(ntotalTest // batch_size) + 1) # +1 takes care of the remaining images if 
                                                                         # ntotalTest // batch_size is not a whole number
# for every image in the testing set, find the the index of the label that has 
# the corresponding highest predicted probability
pred_Idxs = np.argmax(predictions, axis=1)

# set target names
target_names = target_names


# display a well-formated classification results
print(classification_report(test_generator.classes, pred_Idxs,                            
                            target_names = target_names)) 

## Compute the Confusion Matrix

In [ ]:
# define the Confusion Matrix

CM = confusion_matrix(test_generator.classes, pred_Idxs)


In [ ]:
def plot_confusion_matrix(CM, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.    
    """
    
  # plt.figure(figsize=[10,10]) # set the size of the confusion matrix if default is too small 
    plt.imshow(CM, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tickmarks = np.arange(len(classes))
    plt.xticks(tickmarks, classes, rotation=45)
    plt.yticks(tickmarks, classes)    
        
    if normalize:
        CM = CM.astype('float') / CM.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(CM)
    
    fmt = '.2f' if normalize else 'd'
    thresh = CM.max() / 2.
    for i, j in itertools.product(range(CM.shape[0]), range(CM.shape[1])):
        plt.text(j, i, format(CM[i, j], fmt),
                    horizontalalignment="center", verticalalignment="center",
                    color="white" if CM[i, j] > thresh else "black")
        
    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label') 

## Plot Confusion Matrix Without Normalization

In [ ]:
# print network name and the crop name
print('{}{}{}{}{}'.format("The Confusion Matrix below is for: ",CropName, " trained on:", Network_name, " Network"))
print()   

print(CropName,Network_name)
print()

# plot non-normalized confusion matrix
plot_confusion_matrix(CM, classes=target_names,
                      title='Confusion matrix, without normalization')

## Plot Normalized Confusion Matrix 

In [ ]:
# print network name and the crop name
print('{}{}{}{}{}'.format("The Confusion Matrix below is for: ",CropName, " trained on:", Network_name, " Network"))
print()   

print(CropName,Network_name)
print()

# plot normalized confusion matrix
plot_confusion_matrix(CM, classes=target_names, normalize=True,
                      title='Normalized confusion matrix')

## Summary Evaluation on Test Data

In [ ]:
print("[INFO] Now evaluating the trained network...")

# summarize the evaluation of the model on test data

test_loss, test_acc = model.evaluate_generator(test_generator, steps=ntotalTest // batch_size)
print('Test accuracy of the model is:', test_acc)
print('Test loss of the model is:', test_loss)

## Unfreeze the Convolutional Base and Freeze Some Layers Inside this Base


In this step, if for example we are fine-tuning a pre-trained VGG16 network, we may fine-tune the last 3 convolutional layers of the VGG16 network. This means that all the layers up to block4_pool will be frozen and the layers: block5_conv1, block5_conv2, and block5_conv3 will now be trainable. It is advisable to only fine-tune the top convolutional layers since these are the layers that encode more specialized features whereas the initial layers encode more generic features. We should not fine-tune many layers because the more the parameters we train, the more we RISK overfitting the model. This process should be repeated with other pre-trained networks. The number of layers to unfreeze may, for example, be selected to ensure that the number of unfrozen parameters is roughly the same for the different networks to ensure fair comparison of the networks' performances.  

At this point, the top layers are well trained and we can start fine-tuning convolutional layers from VGG16/other pre-trained networks. We will freeze the bottom N layers
and train the remaining top layers.

In [ ]:
# the code here lets us visualize layer names and layer indices to help us determine the number of layers
# we should freeze:
for i, layer in enumerate(base.layers):
   print(i, layer.name)

In [ ]:
# now let's visualize layer names and layer indices in the whole model before unfreezing the conv base:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

In Keras, each layer has a parameter called trainable. To freeze the weights of a particular layer, the trainable parameter is set to False to indicate that that particular  layer should not be trained. 

## Unfreeze the Convolutional Base

In [ ]:
print('This is the number of trainable weights before unfreezing the convolutional base:', len(model.trainable_weights))

for layer in base.layers:
    layer.trainable = True

print('This is the number of trainable weights after unfreezing the convolutional base:', len(model.trainable_weights))

In [ ]:
# We can now check the trainable status of the individual layers
# in the whole model
for i, layer in enumerate(model.layers):
   print(i, layer.trainable)

## Freeze Some Layers Inside the Convolutional Base
Only higher layers of the network should be fine-tuned because they are the ones that encode the more specialized features of the custom problem. Fine-tuning lower layers is not only costly, it also increases the risk of overfitting because more parameters will be trained with the small custom dataset.

In [ ]:
print("[REMINDER]The network we are working with is:  ", Network_name)

In [ ]:
# this function determines the nnumber of layers to freeze for each different network
# these layer numbers can be adjusted e.g., to ensure that the number of frozen layers 
# have roughly the same number of trainable parameters if our goal is to compare the performance of 
# different fine-tuned networks

def freezer(Network_name):
    if Network_name == "vgg16":
        for layer in model.layers[:15]:   
            layer.trainable = False       
        for layer in model.layers[15:]:   
            layer.trainable = True        # No. of parameters fine-tuned = 7,079,424
            
    elif Network_name == "vgg19":
        for layer in model.layers[:18]:   
            layer.trainable = False
        for layer in model.layers[18:]:   
            layer.trainable = True        # No. of parameters fine-tuned = 7,079,424

    elif Network_name == "resnet50":
        for layer in model.layers[:157]:   
            layer.trainable = False
        for layer in model.layers[157:]:   
            layer.trainable = True         # No. of parameters fine-tuned = 7,892,480

    elif Network_name == "inception":
        for layer in model.layers[:261]:  
            layer.trainable = False
        for layer in model.layers[261:]:  
            layer.trainable = True        # No. of parameters fine-tuned = 7,216,704

    elif Network_name == "xception":
        for layer in model.layers[:113]:  
            layer.trainable = False
        for layer in model.layers[113:]:  
            layer.trainable = True        # No. of parameters fine-tuned = 7,340,552

 

In [ ]:
print('The number of trainable weights before freezing the last 4 (N) layers in the convolutional base is:', len(model.trainable_weights))

freezer(Network_name)

print('And the number of trainable weights after freezing the the last 4 layers in the convolutional base is:', len(model.trainable_weights))


In [ ]:
# We can now check the trainable status of the individual layers
# in the convolutional base
for layer in base.layers:
    print(layer, layer.trainable)

In [ ]:
# let's visualize trainable layers names and layer indices in the whole model after freezing some layers in the conv base:
for i, layer in enumerate(model.layers):
   print(i, layer.trainable)

## Re-Compile the model

To avoid errors, recompile the model before printing the summary for the 'trainable' changes to take effect

In [ ]:
# compile the model to ensure the 'trainable' changes have taken effect
# the model should always be re-compiled whenever weight trainability is changed after
# the initial compilation, otherwise the changes will be ignored
# here we can use a lower learning rate than the one used in the earlier training
# using a lower learning rate limits the size of the changes made to the representations 
# of the layers being fine-tuned because large updates will harm the representations 
# already learned

model.compile(optimizer=optimizers.Adam(lr=1e-5),  # now use a lower learning rate
              loss='categorical_crossentropy',
              metrics=['acc'])

## Model Summary

In [ ]:
# now check the number of trainable parameters (for fine tuning)
# we can use this to ensure roughly the same number of parameters are fine-tuned when we compare networks
# 
model.summary()

## Train the model

## Reset Training Parameters

In [ ]:
# set the batch size
batch_size = 30

# set the number of epochs
epochs = 15                   # increase the number of epochs for the fine-tuning process
   

In [ ]:
# %%time # optionallt time the training process
history = model.fit_generator(train_generator,
                    steps_per_epoch=ntotalTrain // batch_size,
                    epochs=epochs,
                    validation_data= val_generator,
                    validation_steps=ntotalVal // batch_size,
                    class_weight=class_weights)

## Save the Model

In [ ]:
model.save("C:\\Path\\to\\TrainedModel3.h5")

# You can also save model without the optimizer to avoid the optimizer error shown when loading the trained model later or
# in a different environment

model.save("C:\\Path\\to\\TrainedModel_4.h5, include_optimizer= False")


## Plot the Training Results

In [ ]:
# This code will plot the curves of loss and accuracy during training

# Get values that were specified during model compilation which are saved in the 
# history object
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get the number of epochs from the values in the 'acc' list
epochs = range(1, len(acc) + 1)

# Training and validation accuracy plot [Accuracy at each epoch]
plt.plot(epochs, acc, 'b', label='Training acc')       
plt.plot(epochs, val_acc, 'r', label='Validation acc')  
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='center right')
plt.figure()

# Training and validation loss plot [Loss at each epoch]
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='center right')
plt.show()


## Evaluate Model on the Testing Dataset/ Make Inferences from Unseen Data

In [ ]:
# First, reset the testing generator before using the trained model to make predictions
# on the test data
print("[INFO] Now evaluating the trained network...")
test_generator.reset()
predictions = model.predict_generator(test_generator,
                                   steps=(ntotalTest // batch_size) + 1)  
                                                                         
# for every image in the testing set, find the the index of the label that has 
# the corresponding highest predicted probability
pred_Idxs = np.argmax(predictions, axis=1) # get the highest prediction indices for each sample 

# set target names
target_names = target_names

# print a well-formated classification results
print(classification_report(test_generator.classes, pred_Idxs,                            
                            target_names = target_names)) 

## Compute the Confusion Matrix

In [ ]:
# define the Confusion Matrix

CM = confusion_matrix(test_generator.classes, pred_Idxs)


In [ ]:
def plot_confusion_matrix(CM, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.    
    """
    
  # plt.figure(figsize=[10,10]) # set the size of the confusion matrix if default is too small
    plt.imshow(CM, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tickmarks = np.arange(len(classes))
    plt.xticks(tickmarks, classes, rotation=45)
    plt.yticks(tickmarks, classes)    
        
    if normalize:
        CM = CM.astype('float') / CM.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(CM)
    
    fmt = '.2f' if normalize else 'd'
    thresh = CM.max() / 2.
    for i, j in itertools.product(range(CM.shape[0]), range(CM.shape[1])):
        plt.text(j, i, format(CM[i, j], fmt),
                    horizontalalignment="center", verticalalignment="center",
                    color="white" if CM[i, j] > thresh else "black")
        
    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label') 

## Plot the Confusion Matrix Without Normalization

In [ ]:
# print network name and the crop name
print('{}{}{}{}{}'.format("The Confusion Matrix below is for: ",CropName, " trained on:", Network_name, " Network"))
print()   

print(CropName,Network_name)
print()

# plot non-normalized confusion matrix
plot_confusion_matrix(CM, classes=target_names,
                      title='Confusion matrix, without normalization')

## Plot the Normalized Confusion Matrix

In [ ]:
# print network name and the crop name
print('{}{}{}{}{}'.format("The Confusion Matrix below is for: ",CropName, " trained on:", Network_name, " Network"))
print()   

print(CropName,Network_name)
print()

# plot normalized confusion matrix
plot_confusion_matrix(CM, classes=target_names, normalize=True,
                      title='Normalized confusion matrix')

## Summary Evaluation on Test Data

In [ ]:
print("[INFO] Now evaluating the trained network...")

# summarize the evaluation of the model on test data

test_loss, test_acc = model.evaluate_generator(test_generator, steps=ntotalTest // batch_size)
print('Test accuracy of the model is:', test_acc)
print('Test loss of the model is:', test_loss)

## Visualize the Results

In [ ]:
# the following code uses the test generator to visualize the errors made in the testing set
 
# get the filenames from the generator
fnames = test_generator.filenames
 
# get the ground truth from generator
ground_truth = test_generator.classes
 
# get the label to class mapping from the generator
label2index = test_generator.class_indices
 
# now get the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 

#predicted_classes = np.argmax(predictions,axis=1) 
errors = np.where(pred_Idxs != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),test_generator.samples))
 
# Show the errors
for i in range(len(errors)):
    pred_class = np.argmax(pred_Idxs[errors[i]])
    pred_label = idx2label[pred_class]
     
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        pred_Idxs[errors[i]][pred_class])
     
    original = load_img('{}/{}'.format(TEST_PATH, fnames[errors[i]]))
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()


## Unlock the Kernel Lock on GPU

In [ ]:
# if you execute the notebook and leave it running (especially when working with multiple notebooks),
# the following code will release the kernel lock on the GPU and help avoid the raising of "resource exhausted"
# errors

%%javascript 
Jupyter.notebook.session.delete();